<a href="https://colab.research.google.com/github/michhottinger/DS-Unit-1-Sprint-2-Statistics/blob/master/1_Michelle_Hottinger_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [111]:
### YOUR CODE STARTS HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-10-07 23:14:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.3’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2019-10-07 23:14:17 (592 KB/s) - ‘house-votes-84.data.3’ saved [18171/18171]



In [112]:
import pandas as pd
import numpy as np
df = pd.read_csv('house-votes-84.data',
                 header=None,
                 names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])
df.shape
df.head(10)

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
9,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [113]:
df= df.replace({'?':np.NaN, 'n':0, 'y':1})

df.head(10)

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
9,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [114]:
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [115]:
rep = df[df.party == "republican"]
print(rep.shape)
rep.head()

(168, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [116]:
df.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [117]:
rep.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [118]:
dem = df[df.party == "democrat"]
print(dem.shape)
dem.head(10)

(267, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0
9,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
12,democrat,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN
13,democrat,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,0.0,0.0,1.0,NaN
16,democrat,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,1.0,1.0,1.0,NaN,0.0,0.0,1.0
17,democrat,1.0,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [119]:
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats
rep['handicapped-infants'].mean()

0.18787878787878787

In [120]:
dem['handicapped-infants'].mean()

0.6046511627906976

In [121]:
ttest_1samp(rep['handicapped-infants'], 0, nan_policy='omit')

Ttest_1sampResult(statistic=6.159569669016066, pvalue=5.434587970316366e-09)

In [122]:
ttest_1samp(rep['physician-fee-freeze'], 1, nan_policy='omit')

Ttest_1sampResult(statistic=-1.4185450076223511, pvalue=0.1579292482594923)

In [123]:
#2 sample test uses means of two samples
ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'],
          nan_policy='omit')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [124]:
#water-project for dems and reps
ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')
#large pvalue, small statistic. p > 0.1 so this says that there is a high chance that the data is as extreme as the null and it cannot be rejected.

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [125]:
dem['water-project'].mean()

0.502092050209205

In [126]:
rep['water-project'].mean()

0.5067567567567568

As seen above, both dems and reps have a similar average on the Water Project.

In [127]:
#The tiny p value means we can reject the null hyp of equal averages for the two groups
ttest_ind(rep['religious-groups'], dem['religious-groups'], nan_policy='omit')

Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

In [128]:
rep['religious-groups'].mean()

0.8975903614457831

In [129]:
ttest_1samp(rep['religious-groups'], 1, nan_policy='omit')

Ttest_1sampResult(statistic=-4.338836636208457, pvalue=2.488389920449274e-05)

In [130]:
dem['religious-groups'].mean()

0.47674418604651164

As seen above, the numbers are very different here


In [131]:
#write a func

def ttest(column_name):
  return ttest_ind(rep[column_name], dem[column_name], nan_policy='omit')

ttest("religious-groups")

Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

In [133]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

In [0]:
import io
df1 = pd.read_csv(io.BytesIO(uploaded['HNP_StatsCountry.csv']))

In [0]:
df1.shape

In [0]:
df1.head(10) #might be interesting

In [135]:
from google.colab import files
uploaded = files.upload()

Saving HNP_StatsCountry-Series.csv to HNP_StatsCountry-Series (1).csv
Saving HNP_StatsCountry.csv to HNP_StatsCountry (2).csv
Saving HNP_StatsData.csv to HNP_StatsData (1).csv
Saving HNP_StatsFootNote.csv to HNP_StatsFootNote (1).csv
Saving HNP_StatsSeries-Time.csv to HNP_StatsSeries-Time (1).csv
Saving HNP_StatsSeries.csv to HNP_StatsSeries (1).csv


In [0]:
stats_time = pd.read_csv(io.BytesIO(uploaded['HNP_StatsSeries-Time.csv'])) #don't need
stats_data = pd.read_csv(io.BytesIO(uploaded['HNP_StatsData.csv']))
stats_series = pd.read_csv(io.BytesIO(uploaded['HNP_StatsSeries.csv'])) #not interesting

In [137]:
stats_data.head(5) #fert rates by country
stats_data.shape

(104377, 64)

In [138]:
stats_data.head(5)
#SP.DYN.TFRT.IN is code for fert rates

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,134.772341,135.343859,136.030214,135.898025,135.721826,135.387563,134.661214,133.779002,131.298708,128.85221,126.533802,124.318306,122.271111,120.012109,117.734781,115.37723,112.928422,110.411373,106.545709,102.733961,99.00478,95.382601,91.80323,88.880106,85.938672,82.975239,79.944952,76.911034,74.84941,72.819383,71.139649,69.068292,67.332381,65.348866,63.373094,61.239881,59.26089,57.269148,56.254278,55.281521,54.346048,53.309407,52.297743,51.897824,51.538384,51.212963,50.839601,50.519539,50.305287,50.119205,49.966897,49.866891,49.790826,49.344963,48.853665,48.279737,47.505489,46.688851,NaN,NaN
1,Arab World,ARB,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,Adults (ages 15+) living with HIV,SH.DYN.AIDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,Adults (ages 15+) newly infected with HIV,SH.HIV.INCD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
stats_data.dropna(axis=0, how='all')

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,134.772341,135.343859,136.030214,135.898025,135.721826,135.387563,134.661214,133.779002,131.298708,128.852210,126.533802,124.318306,1.222711e+02,1.200121e+02,1.177348e+02,1.153772e+02,1.129284e+02,1.104114e+02,1.065457e+02,1.027340e+02,9.900478e+01,9.538260e+01,9.180323e+01,8.888011e+01,8.593867e+01,8.297524e+01,7.994495e+01,7.691103e+01,7.484941e+01,7.281938e+01,7.113965e+01,6.906829e+01,6.733238e+01,6.534887e+01,6.337309e+01,6.123988e+01,5.926089e+01,5.726915e+01,5.625428e+01,5.528152e+01,5.434605e+01,5.330941e+01,5.229774e+01,5.189782e+01,5.153838e+01,5.121296e+01,5.083960e+01,5.051954e+01,5.030529e+01,5.011921e+01,4.996690e+01,4.986689e+01,4.979083e+01,4.934496e+01,4.885366e+01,4.827974e+01,4.750549e+01,4.668885e+01,NaN,NaN
1,Arab World,ARB,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,Adults (ages 15+) living with HIV,SH.DYN.AIDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,Adults (ages 15+) newly infected with HIV,SH.HIV.INCD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,ARB,"Age at first marriage, female",SP.DYN.SMAM.FE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Arab World,ARB,"Age at first marriage, male",SP.DYN.SMAM.MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,88.061110,89.489513,90.782451,91.898948,92.728007,93.201290,94.087535,94.485922,94.544610,94.434251,94.226666,94.523203,9.459166e+01,9.448229e+01,9.424714e+01,9.393769e+01,9.380399e+01,9.355994e+01,9.322482e+01,9.279264e+01,9.225752e+01,9.196565e+01,9.153506e+01,9.099537e+01,9.038313e+01,8.971913e+01,8.944384e+01,8.902855e+01,8.851707e+01,8.791453e+01,8.733205e+01,8.651017e+01,8.583044e+01,8.470423e+01,8.343262e+01,8.182085e+01,8.054492e+01,7.919867e+01,7.781701e+01,7.642278e+01,7.501733e+01,7.360575e+01,7.218210e+01,7.077012e+01,6.940035e+01,6.809882e+01,6.690233e+01,6.578431e+01,6.477850e+01,6.391383e+01,6.320162e+01,6.257233e+01,6.208933e+01,6.173076e+01,6.144653e+01,6.119531e+01,6.128362e+01,6.125395e+01,6.116547e+01,NaN
8,Arab World,ARB,"Age dependency ratio, old",SP.POP.DPND.OL,6.591951,6.700903,6.793279,6.867327,6.919224,6.947790,7.015592,7.053468,7.070116,7

In [0]:
stats_data[stats_data['Indicator Code'] == 'SP.DYN.TFRT.IN'].head(20)
stats_fert = stats_data[stats_data['Indicator Code'] == 'SP.DYN.TFRT.IN']

In [141]:
stats_fert.shape
stats_fert.head(40)
stats_fert.shape

(259, 64)

In [160]:
fert = stats_fert.drop(['2018', 'Unnamed: 63', 'Country Code', 'Indicator Name', 'Indicator Code'], axis=1)
fert.shape
fert.dtypes

Country Name     object
1960            float64
1961            float64
1962            float64
1963            float64
1964            float64
1965            float64
1966            float64
1967            float64
1968            float64
1969            float64
1970            float64
1971            float64
1972            float64
1973            float64
1974            float64
1975            float64
1976            float64
1977            float64
1978            float64
1979            float64
1980            float64
1981            float64
1982            float64
1983            float64
1984            float64
1985            float64
1986            float64
1987            float64
1988            float64
1989            float64
1990            float64
1991            float64
1992            float64
1993            float64
1994            float64
1995            float64
1996            float64
1997            float64
1998            float64
1999            float64
2000            

In [161]:
fert.tail(5)

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
102479,Virgin Islands (U.S.),5.615,5.651,5.679,5.696,5.698,5.679,5.634,5.561,5.459,5.328,5.167,4.976,4.760,4.527,4.286,4.047,3.818,3.607,3.420,3.261,3.138,3.053,3.000,2.973,2.967,2.973,2.986,2.997,2.999,2.987,2.954,2.895,2.812,2.712,2.599,2.481,2.367,2.266,2.185,2.129,2.060,1.900,2.210,2.070,2.170,2.240,2.030,2.510,2.440,2.370,2.300,2.160,2.050,2.070,2.090,2.110,2.090,2.080
102882,West Bank and Gaza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.718,6.674,6.607,6.515,6.396,6.251,6.085,5.907,5.727,5.550,5.383,5.231,5.094,4.973,4.867,4.776,4.699,4.631,4.567,4.504,4.437,4.360,4.272,4.174,4.067,3.955,3.844,3.738
103285,"Yemen, Rep.",7.938,7.962,7.991,8.026,8.067,8.114,8.166,8.222,8.278,8.334,8.386,8.432,8.471,8.504,8.531,8.554,8.578,8.605,8.636,8.671,8.710,8.752,8.793,8.828,8.853,8.864,8.858,8.833,8.786,8.713,8.606,8.459,8.272,8.048,7.795,7.525,7.251,6.986,6.738,6.514,6.313,6.130,5.954,5.778,5.601,5.425,5.253,5.090,4.940,4.801,4.674,4.554,4.439,4.326,4.214,4.103,3.994,3.890
103688,Zambia,7.115,7.169,7.214,7.249,7.274,7.291,7.304,7.317,7.332,7.349,7.367,7.383,7.392,7.393,7.382,7.359,7.323,7.276,7.219,7.156,7.087,7.017,6.946,6.877,6.810,6.746,6.685,6.624,6.563,6.502,6.442,6.384,6.329,6.278,6.231,6.189,6.153,6.121,6.093,6.065,6.036,6.003,5.963,5.915,5.859,5.794,5.724,5.650,5.574,5.496,5.415,5.328,5.233,5.132,5.026,4.918,4.814,4.718
104091,Zimbabwe,7.158,7.215,7.267,7.311,7.347,7.373,7.391,7.403,7.411,7.415,7.419,7.425,7.434,7.441,7.442,7.427,7.385,7.308,7.195,7.045,6.865,6.663,6.451,6.240,6.034,5.836,5.642,5.448,5.252,5.055,4.862,4.676,4.503,4.346,4.207,4.088,3.989,3.907,3.839,3.786,3.748,3.725,3.718,3.725,3.744,3.775,3.819,3.873,3.931,3.988,4.034,4.059,4.058,4.030,3.974,3.896,3.804,3.707


In [162]:
euro = fert[fert['Country Name'] == "Euro area"]
euro.head(5)

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
2938,Euro area,2.600935,2.632849,2.638521,2.679861,2.733397,2.679485,2.66122,2.619847,2.557253,2.487126,2.39322,2.344505,2.235837,2.131895,2.099879,2.023536,1.982763,1.908451,1.852953,1.801117,1.774395,1.726226,1.698099,1.639297,1.601979,1.590983,1.579788,1.558214,1.562587,1.529803,1.534274,1.488901,1.46777,1.435145,1.39783,1.383083,1.395901,1.415001,1.415907,1.431339,1.470893,1.456743,1.459447,1.473768,1.49632,1.500341,1.523665,1.540724,1.581154,1.56445,1.580368,1.562092,1.55441,1.530457,1.553311,1.54983,1.566814,1.561489


In [163]:
usa = fert[fert['Country Name'] == "United States"]
usa.head(5)
usa.describe()
#why is std column all NaN? it must equal zero!

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
count,1.000,1.00,1.000,1.000,1.00,1.000,1.000,1.000,1.000,1.000,1.00,1.000,1.00,1.000,1.000,1.000,1.000,1.00,1.00,1.000,1.0000,1.000,1.0000,1.000,1.0000,1.000,1.0000,1.000,1.000,1.000,1.000,1.0000,1.000,1.0000,1.0000,1.000,1.000,1.000,1.000,1.0000,1.000,1.0000,1.0000,1.0000,1.0000,1.000,1.000,1.00,1.000,1.000,1.000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
mean,3.654,3.62,3.461,3.319,3.19,2.913,2.721,2.558,2.464,2.456,2.48,2.266,2.01,1.879,1.835,1.774,1.738,1.79,1.76,1.808,1.8395,1.812,1.8275,1.799,1.8065,1.844,1.8375,1.872,1.934,2.014,2.081,2.0625,2.046,2.0195,2.0015,1.978,1.976,1.971,1.999,2.0075,2.056,2.0305,2.0205,2.0475,2.0515,2.057,2.108,2.12,2.072,2.002,1.931,1.8945,1.8805,1.8575,1.8625,1.8435,1.8205,1.7655
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,3.654,3.62,3.461,3.319,3.19,2.913,2.721,2.558,2.464,2.456,2.48,2.266,2.01,1.879,1.835,1.774,1.738,1.79,1.76,1.808,1.8395,1.812,1.8275,1.799,1.8065,1.844,1.8375,1.872,1.934,2.014,2.081,2.0625,2.046,2.0195,2.0015,1.978,1.976,1.971,1.999,2.0075,2.056,2.0305,2.0205,2.0475,2.0515,2.057,2.108,2.12,2.072,2.002,1.931,1.8945,1.8805,1.8575,1.8625,1.8435,1.8205,1.7655
25%,3.654,3.62,3.461,3.319,3.19,2.913,2.721,2.558,2.464,2.456,2.48,2.266,2.01,1.879,1.835,1.774,1.738,1.79,1.76,1.808,1.8395,1.812,1.8275,1.799,1.8065,1.844,1.8375,1.872,1.934,2.014,2.081,2.0625,2.046,2.0195,2.0015,1.978,1.976,1.971,1.999,2.0075,2.056,2.0305,2.0205,2.0475,2.0515,2.057,2.108,2.12,2.072,2.002,1.931,1.8945,1.8805,1.8575,1.8625,1.8435,1.8205,1.7655
50%,3.654,3.62,3.461,3.319,3.19,2.913,2.721,2.558,2.464,2.456,2.48,2.266,2.01,1.879,1.835,1.774,1.738,1.79,1.76,1.808,1.8395,1.812,1.8275,1.799,1.8065,1.844,1.8375,1.872,1.934,2.014,2.081,2.0625,2.046,2.0195,2.0015,1.978,1.976,1.971,1.999,2.0075,2.056,2.0305,2.0205,2.0475,2.0515,2.057,2.108,2.12,2.072,2.002,1.931,1.8945,1.8805,1.8575,1.8625,1.8435,1.8205,1.7655
75%,3.654,3.62,3.461,3.319,3.19,2.913,2.721,2.558,2.464,2.456,2.48,2.266,2.01,1.879,1.835,1.774,1.738,1.79,1.76,1.808,1.8395,1.812,1.8275,1.799,1.8065,1.844,1.8375,1.872,1.934,2.014,2.081,2.0625,2.046,2.0195,2.0015,1.978,1.976,1.971,1.999,2.0075,2.056,2.0305,2.0205,2.0475,2.0515,2.057,2.108,2.12,2.072,2.002,1.931,1.8945,1.8805,1.8575,1.8625,1.8435,1.8205,1.7655
max,3.654,3.62,3.461,3.319,3.19,2.913,2.721,2.558,2.464,2.456,2.48,2.266,2.01,1.879,1.835,1.774,1.738,1.79,1.76,1.808,1.8395,1.812,1.8275,1.799,1.8065,1.844,1.8375,1.872,1.934,2.014,2.081,2.0625,2.046,2.0195,2.0015,1.978,1.976,1.971,1.999,2.0075,2.056,2.0305,2.0205,2.0475,2.0515,2.057,2.108,2.12,2.072,2.002,1.931,1.8945,1.8805,1.8575,1.8625,1.8435,1.8205,1.7655


In [164]:
swe = fert[fert['Country Name'] == "Sweden"]
swe.head(5)
swe.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
count,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00
mean,2.17,2.21,2.25,2.33,2.47,2.39,2.37,2.28,2.07,1.93,1.92,1.96,1.91,1.87,1.87,1.77,1.68,1.64,1.6,1.66,1.68,1.63,1.62,1.61,1.66,1.74,1.8,1.84,1.96,2.01,2.13,2.11,2.09,1.99,1.88,1.73,1.6,1.52,1.5,1.5,1.54,1.57,1.65,1.71,1.75,1.77,1.85,1.88,1.91,1.94,1.98,1.9,1.91,1.89,1.88,1.85,1.85,1.85
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.17,2.21,2.25,2.33,2.47,2.39,2.37,2.28,2.07,1.93,1.92,1.96,1.91,1.87,1.87,1.77,1.68,1.64,1.6,1.66,1.68,1.63,1.62,1.61,1.66,1.74,1.8,1.84,1.96,2.01,2.13,2.11,2.09,1.99,1.88,1.73,1.6,1.52,1.5,1.5,1.54,1.57,1.65,1.71,1.75,1.77,1.85,1.88,1.91,1.94,1.98,1.9,1.91,1.89,1.88,1.85,1.85,1.85
25%,2.17,2.21,2.25,2.33,2.47,2.39,2.37,2.28,2.07,1.93,1.92,1.96,1.91,1.87,1.87,1.77,1.68,1.64,1.6,1.66,1.68,1.63,1.62,1.61,1.66,1.74,1.8,1.84,1.96,2.01,2.13,2.11,2.09,1.99,1.88,1.73,1.6,1.52,1.5,1.5,1.54,1.57,1.65,1.71,1.75,1.77,1.85,1.88,1.91,1.94,1.98,1.9,1.91,1.89,1.88,1.85,1.85,1.85
50%,2.17,2.21,2.25,2.33,2.47,2.39,2.37,2.28,2.07,1.93,1.92,1.96,1.91,1.87,1.87,1.77,1.68,1.64,1.6,1.66,1.68,1.63,1.62,1.61,1.66,1.74,1.8,1.84,1.96,2.01,2.13,2.11,2.09,1.99,1.88,1.73,1.6,1.52,1.5,1.5,1.54,1.57,1.65,1.71,1.75,1.77,1.85,1.88,1.91,1.94,1.98,1.9,1.91,1.89,1.88,1.85,1.85,1.85
75%,2.17,2.21,2.25,2.33,2.47,2.39,2.37,2.28,2.07,1.93,1.92,1.96,1.91,1.87,1.87,1.77,1.68,1.64,1.6,1.66,1.68,1.63,1.62,1.61,1.66,1.74,1.8,1.84,1.96,2.01,2.13,2.11,2.09,1.99,1.88,1.73,1.6,1.52,1.5,1.5,1.54,1.57,1.65,1.71,1.75,1.77,1.85,1.88,1.91,1.94,1.98,1.9,1.91,1.89,1.88,1.85,1.85,1.85
max,2.17,2.21,2.25,2.33,2.47,2.39,2.37,2.28,2.07,1.93,1.92,1.96,1.91,1.87,1.87,1.77,1.68,1.64,1.6,1.66,1.68,1.63,1.62,1.61,1.66,1.74,1.8,1.84,1.96,2.01,2.13,2.11,2.09,1.99,1.88,1.73,1.6,1.52,1.5,1.5,1.54,1.57,1.65,1.71,1.75,1.77,1.85,1.88,1.91,1.94,1.98,1.9,1.91,1.89,1.88,1.85,1.85,1.85


In [165]:
#pick two countries to compare using function from top of page
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats
def ttest(column_name):
  return ttest_ind(usa[column_name], swe[column_name])

ttest("2016")
#this won't work because they aren't "trials" so each country has their own data

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Ttest_indResult(statistic=nan, pvalue=nan)

In [168]:
ttest_ind(fert['2016'], fert['2015'],
          nan_policy='omit')]
#there we go, we need to compare years, not countries to to the ttest
#I need to better understand the math so I apply it properly!

Ttest_indResult(statistic=-0.2111234161424914, pvalue=0.8328803844261299)

In [0]:
#usa.isnull().sum()
#swe.isnull().sum()
#no null values all integers
#Need to find out why ttest is not running on this data